In [1]:
#Dataset

In [34]:
import pandas as pd
import os
import numpy as np

datapath1='Data csv'
Images_data = 'X-ray images dataset'
dataset_path = 'dataset'

categories=os.listdir(dataset_path)
print(categories)

dataset=pd.read_csv(os.path.join(datapath1,'Train_set_covid.csv'))
status = dataset['label']
image_names = dataset['filename']

['Covid Negative', 'Covid Positive']


In [35]:
positive_index = np.concatenate(np.where(status==1))
positive_image_names = image_names[positive_index]

In [36]:
import cv2
for positive_image_name in positive_image_names:
    image = cv2.imread(os.path.join('X-ray images dataset',positive_image_name))
    try : 
        cv2.imwrite(os.path.join(dataset_path,categories[1],positive_image_name),image)
    except Exception as e:
        print(e)

In [37]:
negative_index = np.concatenate(np.where(status==0))
negative_image_names = image_names[negative_index]

In [39]:
import cv2
for negative_image_name in negative_image_names:
    image = cv2.imread(os.path.join('X-ray images dataset',negative_image_name))
    try : 
        cv2.imwrite(os.path.join(dataset_path,categories[0],negative_image_name),image)
    except Exception as e:
        print(e)

# Preprocess##

In [40]:
import cv2,os

data_path='dataset'
categories=os.listdir(data_path)
labels=[i for i in range(len(categories))]

label_dict=dict(zip(categories,labels)) #empty dictionary

print(label_dict)
print(categories)
print(labels)

{'Covid Negative': 0, 'Covid Positive': 1}
['Covid Negative', 'Covid Positive']
[0, 1]


In [41]:
img_size=128 #resizing images
data=[]
target=[]


for category in categories:
    folder_path=os.path.join(data_path,category)
    img_names=os.listdir(folder_path)
        
    for img_name in img_names:
        img_path=os.path.join(folder_path,img_name)
        img=cv2.imread(img_path)

        try:
            gray=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)           
            #Coverting the image into gray scale
            resized=cv2.resize(gray,(img_size,img_size))
            #resizing the gray scale into 100x100, since we need a fixed common size for all the images in the dataset
            data.append(resized)
            target.append(label_dict[category])
            #appending the image and the label(categorized) into the list (dataset)

        except Exception as e:
            print('Exception:',e)
            #if any exception rasied, the exception will be printed here. And pass to the next image

In [42]:
import numpy as np

data=np.array(data)/255.0
data=np.reshape(data,(data.shape[0],img_size,img_size,1))
target=np.array(target)

from keras.utils import np_utils

new_target=np_utils.to_categorical(target)

In [43]:
new_target

array([[1., 0.],
       [1., 0.],
       [1., 0.],
       ...,
       [0., 1.],
       [0., 1.],
       [0., 1.]], dtype=float32)

In [44]:
np.save('data',data)
np.save('target',new_target)

# TRAINING using CNN

In [45]:
import numpy as np

data=np.load('data.npy')
target=np.load('target.npy')

In [46]:
from keras.models import Sequential,Model
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D,Activation,MaxPooling2D
from keras.utils import normalize
from keras.layers import Concatenate
from keras import Input
from keras.callbacks import ModelCheckpoint

input_shape=data.shape[1:] #50,50,1
inp=Input(shape=input_shape)
convs=[]

parrallel_kernels=[3,5,7]

for k in range(len(parrallel_kernels)):

    conv = Conv2D(128, parrallel_kernels[k],padding='same',activation='relu',input_shape=input_shape,strides=1)(inp)
    convs.append(conv)

out = Concatenate()(convs)
conv_model = Model(inp, out)


model = Sequential()
model.add(conv_model)

model.add(Conv2D(64,(3,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(32,(3,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Flatten())
model.add(Dropout(0.5))
model.add(Dense(128,activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64,activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(2,input_dim=128,activation='softmax'))
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
model_1 (Functional)         (None, 128, 128, 384)     11008     
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 126, 126, 64)      221248    
_________________________________________________________________
activation_2 (Activation)    (None, 126, 126, 64)      0         
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 63, 63, 64)        0         
_________________________________________________________________
conv2d_12 (Conv2D)           (None, 61, 61, 32)        18464     
_________________________________________________________________
activation_3 (Activation)    (None, 61, 61, 32)        0         
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 30, 30, 32)       

In [47]:
input_shape

(128, 128, 1)

In [48]:
from sklearn.model_selection import train_test_split

train_data,test_data,train_target,test_target=train_test_split(data,target,test_size=0.2)

In [49]:
train_target.shape

(2783, 2)

In [50]:
checkpoint = ModelCheckpoint('model-{epoch:03d}.model',monitor='val_loss',verbose=0,save_best_only=True,mode='auto')
history=model.fit(train_data,train_target,epochs=5,callbacks=[checkpoint],validation_split=0.2)

Epoch 1/5
 3/70 [>.............................] - ETA: 28:06 - loss: 0.6663 - accuracy: 0.6406

KeyboardInterrupt: 

In [ ]:
print(model.evaluate(test_data,test_target))

In [25]:
ls {checkpoint_dir}

In [ ]:
latest = tf.train.latest_checkpoint(checkpoint_dir)
latest

In [ ]:
# Create a new model instance
model = create_model()

# Load the previously saved weights
model.load_weights(latest)

# Re-evaluate the model
loss, acc = model.evaluate(test_images,  test_labels, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

In [ ]:
#prediction = model.predict(data)

In [ ]:
prediction[0]

In [ ]:
from matplotlib import pyplot as plt

plt.plot(history.history['loss'],'r',label='training loss')
plt.plot(history.history['val_loss'],label='validation loss')
plt.xlabel('# epochs')
plt.ylabel('loss')
plt.legend()
plt.show()

In [ ]:
plt.plot(history.history['accuracy'],'r',label='training accuracy')
plt.plot(history.history['val_accuracy'],label='validation accuracy')
plt.xlabel('# epochs')
plt.ylabel('loss')
plt.legend()
plt.show()

0.16 = MSE 
0.9568 = Accuracy

In [ ]:
# read image
im = Image.open('Image_1.jpg')

X = preprocess(im,input_size)
X = reshape([X])
y = model.predict(X)
print( labels[np.argmax(y)], np.max(y) )

# Using The Model

In [ ]:
import cv2,os

data_path_test='dataset test'
categories=os.listdir(data_path_test)
labels=[i for i in range(len(categories))]

label_dict=dict(zip(categories,labels)) #empty dictionary

print(label_dict)
print(categories)
print(labels)

In [ ]:
img_size=128 #resizing images
data_test=[]

img_names=os.listdir(data_path_test)

for img_name in img_names:
    img_path=os.path.join(data_path_test,img_name)
    img=cv2.imread(img_path)
    try:
        gray=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)           
        #Coverting the image into gray scale
        resized=cv2.resize(gray,(img_size,img_size))
        #resizing the gray scale into 100x100, since we need a fixed common size for all the images in the dataset
        data_test.append(resized)
        #appending the image and the label(categorized) into the list (dataset)
    except Exception as e:
        print('Exception:',e)
            #if any exception rasied, the exception will be printed here. And pass to the next image


In [ ]:
import numpy as np

data_test=np.array(data_test)/255.0
data_test=np.reshape(data_test,(data_test.shape[0],img_size,img_size,1))

from keras.utils import np_utils

In [ ]:
#prediction_test = model.predict(data_test)

In [ ]:
prediction_test[0]

In [ ]:
imagetest = data[0]
imagetest.shape

In [ ]:
imagetest = np.expand_dims(imagetest, axis=2)
imagetest.shape

In [ ]:
#Load Model pickle
loaded_model = pickle.load(open(filename, 'rb'))
result = loaded_model.score(X_test, Y_test)
print(result)

In [ ]:
#Load model joblib
loaded_model = joblib.load(filename)
result = loaded_model.score(X_test, Y_test)
print(result)